In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# Set the random seed for reproducibility
np.random.seed(42)

# Generate 50,000 low temperatures between 0 and 50 degrees (adjust range as needed)
low_temps = np.random.uniform(15, 25, size=50000)

# Generate 50,000 high temperatures between 50 and 100 degrees (adjust range as needed)
high_temps = np.random.uniform(27, 35, size=50000)

In [ ]:
# Create a DataFrame with the generated temperatures
df = pd.DataFrame({'Low Temp': low_temps, 'High Temp': high_temps})

In [ ]:
# Save the DataFrame to a CSV file named 'temperatures.csv'
df.to_csv('temperatures.csv', index=False)

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [ ]:
# Load the dataset
df = pd.read_csv('temperatures.csv')

# Extract the 'Low Temp' column as the target variable
data = df['Low Temp'].values.reshape(-1, 1)

# Scale the data to the range [0, 1]
scaler = MinMaxScaler()
data = scaler.fit_transform(data)

# Split the data into training and testing sets
train_size = int(len(data) * 0.8)
train_data, test_data = data[0:train_size], data[train_size:len(data)]

# Create sequences of data for the LSTM model
def create_dataset(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        X.append(a)
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

look_back = 10
X_train, Y_train = create_dataset(train_data, look_back)
X_test, Y_test = create_dataset(test_data, look_back)

# Reshape the input data to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [ ]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50))
model.add(Dense(1))

# Compile the model
model.compile(loss='mean_squared_error', optimizer='adam')

# Train the model
model.fit(X_train, Y_train, epochs=100, batch_size=32)
model.save('lstm_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 19s 12ms/step - loss: 0.0894
Epoch 2/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 21s 12ms/step - loss: 0.0839
Epoch 3/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 20s 12ms/step - loss: 0.0833
Epoch 4/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 15s 12ms/step - loss: 0.0837
Epoch 5/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 15s 12ms/step - loss: 0.0834
Epoch 6/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 15s 12ms/step - loss: 0.0839
Epoch 7/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 15s 12ms/step - loss: 0.0838
Epoch 8/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 16s 13ms/step - loss: 0.0836
Epoch 9/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 15s 12ms/step - loss: 0.0826
Epoch 10/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 20s 12ms/step - loss: 0.0827
Epoch 11/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 20s 12ms/step - loss: 0.0832
Epoch 12/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 15s 12ms/step - loss: 0.0832
Epoch 13/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 20s 12ms/step - loss: 0.0833
Epoch 14/100
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 15s 12ms/step - 

In [ ]:
model.save('lstm_model.h5')

In [ ]:
# Make predictions on the test data
predictions = model.predict(X_test)

# Invert the scaling to get the actual temperature values
predictions = scaler.inverse_transform(predictions)
Y_test = scaler.inverse_transform([Y_test])

# Evaluate the model
# ... (e.g., calculate RMSE, plot predictions vs. actual values)

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step


In [ ]:
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
import numpy as np

model = load_model('lstm_model.h5')  # Replace with your model path
scaler = MinMaxScaler()  # You might need to load or recreate your scaler

In [ ]:
temp = 24
input_data = scaler.fit_transform(np.array([[temp]]))
input_data = input_data.reshape(1, 1, 1)  # Assuming look_back=1

In [ ]:
prediction = model.predict(input_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 251ms/step


In [ ]:
if prediction < 0.4:  # Adjust threshold as needed (0.5 after scaling)
    label = "LOW"
else:
    label = "HIGH"

print(f"The predicted label for 29 degrees Celsius is: {label}")

The predicted label for 29 degrees Celsius is: HIGH


In [ ]:
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
import numpy as np

def predict_temperature_label(test_temp, threshold=0.3):
    """Predicts the temperature label (LOW or HIGH) based on the test temperature.

    Args:
        test_temp: The test temperature in Celsius.
        threshold: The threshold value for classification (default: 0.5).

    Returns:
        The predicted temperature label (LOW or HIGH).
    """

    # Load the model and scaler
    model = load_model('lstm_model.h5')  # Replace with your model path
    scaler = MinMaxScaler()

    # Preprocess the input
    input_data = scaler.fit_transform(np.array([[test_temp]]))
    input_data = input_data.reshape(1, 1, 1)  # Assuming look_back=1

    # Make prediction
    prediction = model.predict(input_data)[0][0]  # Extract the prediction value

    # Determine the label based on the threshold
    if prediction < threshold:
        label = "LOW"
    else:
        label = "HIGH"

    return label

# Example usage
test_temperature = 33
predicted_label = predict_temperature_label(test_temperature, threshold=0.6)  # Adjust threshold as needed
print(f"The predicted label for {test_temperature} degrees Celsius is: {predicted_label}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 371ms/step
The predicted label for 33 degrees Celsius is: LOW


In [ ]:
model.save('lstm_model.h5')

NameError: name 'model' is not defined